In [2]:
import pandas as Pandas
import numpy as Numpy
import tensorflow.keras as Keras
import sklearn as Sklearn
import tensorflow as Tf
from sklearn import model_selection as Ms
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
from nltk.metrics.distance  import edit_distance
import imblearn
from imblearn.under_sampling import RandomUnderSampler
nltk.download('wordnet')
nltk.download('words')
correct_spellings = set(words.words())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/My Drive/Cais/SentimentAnalysisFolder

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Cais/SentimentAnalysisFolder


In [4]:
rawData = Pandas.read_csv('dataset/data.csv')
rawData.head()

,valence,author,tweet
0,0,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,scotthamilton,is upset that he can't update his Facebook by ...
2,0,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,ElleCTF,my whole body feels itchy and like its on fire
4,0,Karoli,"@nationwideclass no, it's not behaving at all...."


In [71]:
#This Block was created with the intention of multiclass classification with imbalanced data rather than binary


bigTrain, subDataTest = Ms.train_test_split(rawData, test_size=0.985, stratify=rawData['valence'])
#print(len(bigData))
#subDataTrain, subDataTest = Ms.train_test_split(bigData, test_size=0.25, stratify=bigData['valence'])
#subDataTrain = RandomOverSampler(sampling_strategy=0.5).fit_resample(subDataTrain)


y = bigTrain['valence']
bigTrain = bigTrain.drop('valence', axis=1)

subDataTest = subDataTest[:50000]
x = subDataTest["valence"]
subDataTest = subDataTest.drop('valence', axis=1)

sampler = RandomUnderSampler(random_state=100, replacement=False)
subDataTrain, labelsForTrain = sampler.fit_resample(bigTrain, y)
subDataTest, labelsForTest = sampler.fit_resample(subDataTest, x)


subDataTrain.insert(loc=int(0), column = 'valence', value=labelsForTrain)
subDataTest.insert(loc=int(0), column = 'valence', value=labelsForTest)

In [72]:
vectorizer = Keras.layers.TextVectorization(output_mode='int', output_sequence_length=30)
lemmatizer = WordNetLemmatizer()
corpus = []
garbage = [".", "''", "' '", None, '', "''", ',']

for i in subDataTrain['tweet']:
  for word in i.split(' '):
      corpus.append(word)

cp = set(Numpy.loadtxt('english-dictionary.txt', dtype=str))
sp = Numpy.loadtxt('stopwords.txt', dtype=str)
zp = []

for i in cp:
  if i not in sp:
    zp.append(i)
    #zp.append(lemmatizer.lemmatize(i))

zp = Numpy.array(list(zp))

vectorizer.adapt(zp)

zp = set(zp)

vectorizer.get_vocabulary()[:4]

['', '[UNK]', 'organizations', 'operations']

In [81]:
model = Keras.Sequential()
model.add(Keras.layers.Input(shape=(1,), dtype=Tf.string))
model.add(vectorizer)
model.add(
    Keras.layers.Embedding(len(zp) + 2, 5, input_length=30)
)
model.add(Keras.layers.Bidirectional(Keras.layers.LSTM(20, dropout=0.5)))
model.add(Keras.layers.Dense(10, activation='sigmoid'))
model.add(Keras.layers.Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
print(model.summary())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_5 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_12 (Embedding)    (None, 30, 5)             545945    
                                                                 
 bidirectional_12 (Bidirecti  (None, 40)               4160      
 onal)                                                           
                                                                 
 dense_24 (Dense)            (None, 10)                410       
                                                                 
 dense_25 (Dense)            (None, 2)                 22        
                                                                 
Total params: 550,537
Trainable params: 550,537
Non-t

In [82]:
tweets = []
valences = []
for i,j in zip(subDataTrain['tweet'], subDataTrain['valence']):
  z = ''
  i = "".join(char for char in i if char.isalnum() or char is ' ')
  for c in i.split():
    if c.lower() in zp:
      c = lemmatizer.lemmatize(c.lower())
      z = z + ' ' + c
    #else:
      #predictivehacks.com for this segment of code
      #try:
      #  temp = [(edit_distance(c.lower(), w),w) for w in correct_spellings if w[0]==c.lower()[0]]
      #  l = min(temp, key = lambda val:val[0], default='!!')[1]
      #except:
      #  pass
      #end segment
      #if l is not '!':
      #  print(c + ' was translated to ' + l)
      #  z = z + ' ' + lemmatizer.lemmatize(l)
      

  if z is not '': 
    tweets.append(z)
    valences.append(j)

tweets = Numpy.array(tweets)
valences = Numpy.array(valences)

mapping = {0:0, 4:1}

encodedStuff = [ mapping[i] for i in valences ]
encodedStuff = Numpy.array(encodedStuff)



In [83]:
goodStuff = Keras.utils.to_categorical(encodedStuff)
print(len(tweets))
model.fit(tweets, goodStuff, epochs=5, verbose=2,validation_split=0.2)

23128
Epoch 1/5
579/579 - 17s - loss: 0.6434 - accuracy: 0.6385 - val_loss: 0.6227 - val_accuracy: 0.8299 - 17s/epoch - 29ms/step
Epoch 2/5
579/579 - 12s - loss: 0.5563 - accuracy: 0.7152 - val_loss: 0.7134 - val_accuracy: 0.6554 - 12s/epoch - 20ms/step
Epoch 3/5
579/579 - 12s - loss: 0.5170 - accuracy: 0.7477 - val_loss: 0.6666 - val_accuracy: 0.6788 - 12s/epoch - 21ms/step
Epoch 4/5
579/579 - 12s - loss: 0.5010 - accuracy: 0.7598 - val_loss: 0.7983 - val_accuracy: 0.5223 - 12s/epoch - 20ms/step
Epoch 5/5
579/579 - 12s - loss: 0.4911 - accuracy: 0.7663 - val_loss: 0.7150 - val_accuracy: 0.6407 - 12s/epoch - 21ms/step


In [84]:
model.predict(["not bad far from great im not happy"])

array([[0.80363303, 0.1963669 ]], dtype=float32)

In [85]:
tweets = []
valences = []
for i,j in zip(subDataTest['tweet'], subDataTest['valence']):
  z = ''
  for c in i.split():
    if c.lower() in correct_spellings:
      if c.lower() not in sp:
        c = lemmatizer.lemmatize(c.lower())
        z = z + ' ' + c

  if z is not '':  
    tweets.append(z)
    valences.append(j)

tweets = Numpy.array(tweets)
valences = Numpy.array(valences)

stuff = [ mapping[i] for i in valences ]
stuff = Numpy.array(stuff)
stuff = Keras.utils.to_categorical(stuff)


In [86]:
score, acc = model.evaluate(tweets[:50000], stuff[:50000], verbose=2)

1414/1414 - 7s - loss: 0.5838 - accuracy: 0.6912 - 7s/epoch - 5ms/step
